# Optimizing Portfolio - Technical Analysis Predictions

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.ticker as mtick

import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
aspect_ratio = (12, 8)

# 1. Reading the datasets

Functions helpful for technical analysis

In [2]:
def read_price_data(filename, index_col = 0, parse_dates = True):
    """ Function to read the data frame from the csv file"""
    # dayfirst=True makes the input format to be "dd/mm/yyyy"
    df_stock = pd.read_csv(filename, index_col=index_col, parse_dates=parse_dates, dayfirst=False)
    # stripping the csv to display the stock ticker
    stock_name = filename.strip('.csv')
    print(f'Information about the Stock: {stock_name}')
    # Information about Dataframe
    print(df_stock.info())
    print(f'\n')
    print(f'Null elements in the dataframe for stock: {stock_name}')
    # check for nulls
    print(df_stock.isnull().sum())
    print(f'\n')
    print(f'Summary Statistics for the Stock: {stock_name}')
    # summary statistics for the dataframe
    print(df_stock.describe())
    print(f'Dataframe of the Stock: {stock_name}')
    print(f'\n')
    # dataframe
    print(df_stock)
    print(f'\n\n\n')
    return(df_stock)

def plot_and_format_y_axis(data_frame, formatter=ticker.ScalarFormatter(),
                           aspect_ratio=(12, 8), title="", ylabel="", grid=True):
    """ Function to plot the dataframe columns based on the formatter and format the plot using the given fields """
    plt.figure(figsize=aspect_ratio)
    # Plot every column of the dataframe indivisually
    for col in data_frame.columns:
        plt.plot(data_frame.index, data_frame[col], label=col)
    # turn grid ON
    plt.grid(grid)
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel(ylabel)
    #formatting y-axis Get_Current_Axis and use the formatter
    plt.gca().yaxis.set_major_formatter(formatter)
    plt.legend()
    plt.show()

def print_pct_data(data_frame, format_spec="{:.4%}", na_rep="NaN", index_slice=None):
    """ Pretty print the dataframe. Convert the returns to percentages. """
    # Applying the format and na_rep using the Styler
    if index_slice:
        return data_frame.style.format(format_spec, subset=index_slice, na_rep=na_rep)
    else:
        return data_frame.style.format(format_spec, na_rep=na_rep)

def calcSummaryStatistics(returns, annualization_factor=260, print_screen=True, percent_format_string="{:.2%}"):
    """ Calculate the summary statistics from the returns dataframe. Use annualization factor to determine the conversion factor to 
    convert the daily/monthly returns to annual returns, and pretty print the results."""
    # Checking if the first row contains values [Only True if all the columns in first row has values]
    if returns.iloc[0].notna().all():
        # Inserting a row of NaN values at the top of the dataframe with the correct business date
        Nan_date = pd.date_range(end=returns.index[0], periods=2, freq='B')[0]
        new_row = pd.DataFrame(index=[Nan_date], columns=returns.columns)
        returns = pd.concat([new_row, returns])
    
    # This condition is True when one or more columns in the first row contain NaN
    else:
        # Check whether first row is a mixture of NaN and values. This is True only when first row contains atleast one value
        if not returns.iloc[0].isnull().all():
            raise ValueError("First row should contain NaN values for all columns.")

    # Check if there are NaN values in non-first rows, if yes, raise ValueError
    if returns.iloc[1:].isnull().values.any():
        raise ValueError("NaN rows should only be in the first row")
    
    
    # Calculate cumulative returns (NAV)
    cumulative_returns = (1 + returns).cumprod().fillna(1)

    summary_stats = pd.DataFrame([annualization_factor*returns.mean(), 
                              np.sqrt(annualization_factor)*returns.std(),
                              np.sqrt(annualization_factor) * returns.mean() / returns.std(),
                              (cumulative_returns - np.maximum.accumulate(cumulative_returns)).min(axis = 0),
                              (returns.dropna() >= 0).sum() / returns.dropna().shape[0]],
                              index=['Average Annual Return', 'Annual Risk', 'Sharpe Ratio', 'maxDD', 'Success Ratio'])
    
    if print_screen:
        print("Summary Statistics for Returns:")
        display(print_pct_data(summary_stats, 
                               format_spec=percent_format_string,
                               index_slice = pd.IndexSlice[['Average Annual Return', 'Annual Risk', 'maxDD', 'Success Ratio'], :]))
    else:
        return summary_stats
        
        
def movingAverageCrossover(stock_data, fastWindow, slowWindow):
    """ Takes input of single instrument. This function will implement 3 strategies, one which goes flat moving average, 
    one which goes short moving average, and one with the original daily returns (Benchmark). It calculates the returns
    for the moving average strategies using the fastWindow, and the slowWindow provided in the funciton input"""
    # Error checking
    if len(stock_data.columns) != 1:
        raise ValueError("Input DataFrame should have only one column.")
    
    # Get the ticker name
    stock_name = stock_data.columns[0]
    # Set up the slow and fast MA
    stock_data.loc[:,'FastSMA'] = stock_data[stock_name].rolling(fastWindow).mean()
    stock_data.loc[:,'SlowSMA'] = stock_data[stock_name].rolling(slowWindow).mean()
    
    # Since there are many leading NaN's (because of the MA lookback windows), drop them all.
    stock_data.dropna(inplace=True)
    # Calculating stock price log returns
    stock_data.loc[:,stock_name + '-BMK'] = np.log(stock_data[stock_name] / stock_data[stock_name].shift(1))

    # Go flat strategy
    stock_data['Signal_Flat'] = np.where(stock_data['FastSMA'] > stock_data['SlowSMA'], 1, 0)
    stock_data[stock_name + '-MAFlat'] = stock_data[stock_name + '-BMK'] * stock_data['Signal_Flat'].shift()
    # Go short strategy
    stock_data['Signal_Short'] = np.where(stock_data['FastSMA'] < stock_data['SlowSMA'], -1, 1)
    stock_data[stock_name + '-MAShort'] = stock_data[stock_name + '-BMK'] * stock_data['Signal_Short'].shift()

    strategy_df = stock_data[[stock_name + '-BMK', stock_name + '-MAFlat', stock_name + '-MAShort']]
    return strategy_df


def bollingerBands(stock_data, lookbackWindow, stdevBand):
    """ Takes input of single instrument, a lookbackWindow, and a stdevBand which specifies the # of standard deviations 
    around the mean when the strategy goes “counter-trend.” This function then implements the Bollinger band strategy & 
    returns a single dataFrame with two columns of daily returns – one for the original price series (the “Benchmark”), 
    and a second for the Bollinger Band strategy"""
    # Error checking
    if len(stock_data.columns) != 1:
        raise ValueError("Input DataFrame should have only one column.")
    
    # Get the ticker name
    stock_name = stock_data.columns[0]

    # Calculate the rolling mean and standard deviation
    stock_data.loc[:,'Mean'] = stock_data[stock_name].rolling(window=lookbackWindow).mean()
    stock_data.loc[:,'Stdev'] = stock_data[stock_name].rolling(window=lookbackWindow).std()
    stock_data.dropna(inplace=True)

    # Making Upper and Lower bands
    stock_data.loc[:,'Upper'] = stock_data['Mean'] + stdevBand * stock_data['Stdev']
    stock_data.loc[:,'Lower'] = stock_data['Mean'] - stdevBand * stock_data['Stdev']

    # Implementing Bollinger band based strategy
    date0 = stock_data.index[0]
    if stock_data.loc[date0, stock_name] >= stock_data.loc[date0, 'Upper']:
        stock_data.loc[date0, 'Position'] = -1
    elif stock_data.loc[date0, stock_name] <= stock_data.loc[date0,'Lower']:
        stock_data.loc[date0, 'Position'] = 1
    else:
        stock_data.loc[date0, 'Position'] = 0

    # loop over the entire data set
    for i in range(1, stock_data.shape[0]):
        today = stock_data.index[i]
        yesterday = stock_data.index[i-1]
        if stock_data.loc[today, stock_name] >= stock_data.loc[today, 'Upper']: # if close is above upper
            stock_data.loc[today,'Position'] = -1 # then go short
        elif stock_data.loc[today, stock_name] <= stock_data.loc[today,'Lower']: # if close is below lower
            stock_data.loc[today, 'Position'] = 1 # then go long
        elif stock_data.loc[yesterday, 'Position'] == -1 and stock_data.loc[today, stock_name] <= stock_data.loc[today,'Mean']: # if prev day is short and we're now below the mean
            stock_data.loc[today,'Position'] = 0 # then flatten
        elif stock_data.loc[yesterday,'Position'] == 1 and stock_data.loc[today, stock_name] >= stock_data.loc[today, 'Mean']: # conversely...
            stock_data.loc[today,'Position'] = 0 # then also flatten
        else: # otherwise just hold yesterday's position
            stock_data.loc[today,'Position'] = stock_data.loc[yesterday,'Position']

    # first, calculate SPY stock price log returns
    stock_data.loc[:,stock_name + '-BMK'] = np.log(stock_data[stock_name] / stock_data[stock_name].shift(1))
    # today's Bollinger return = last night's Bollinger position times today's Close Price return
    stock_data.loc[:,stock_name + '-BB'] = stock_data['Position'].shift(1) * stock_data[stock_name + '-BMK']
    strategy_df = stock_data[[stock_name + '-BMK', stock_name + '-BB']]
    
    return strategy_df

In [3]:
# Reading dataframes
dfPrices = read_price_data('PricesThruSep2023.csv', 0, True)

Information about the Stock: PricesThruSep2023
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5975 entries, 1999-12-31 to 2023-09-29
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    5975 non-null   float64
 1   CAD     5975 non-null   float64
 2   EXC     5975 non-null   float64
 3   FBNDX   5975 non-null   float64
 4   GBP     5975 non-null   float64
 5   GE      5975 non-null   float64
 6   INTC    5975 non-null   float64
 7   PFE     5975 non-null   float64
 8   SPGSCI  5975 non-null   float64
 9   SPY     5975 non-null   float64
 10  VBTIX   5975 non-null   float64
 11  XPT     5975 non-null   float64
dtypes: float64(12)
memory usage: 606.8 KB
None


Null elements in the dataframe for stock: PricesThruSep2023
AAPL      0
CAD       0
EXC       0
FBNDX     0
GBP       0
GE        0
INTC      0
PFE       0
SPGSCI    0
SPY       0
VBTIX     0
XPT       0
dtype: int64


Summary Statistics for the Stock: PricesT

# 2. Making the same portfolio again

In [4]:
symbols_strategies = [('AAPL', 0), ('GE', 1), ('SPY', 1), ('FBNDX', 0), ('XPT', 1), ('GBP', 1)]
gf = '-MAFlat'
gs = '-MAShort'
df_crossover1= movingAverageCrossover(pd.DataFrame(dfPrices.loc[:, symbols_strategies[0][0]].copy()),
                                      22, 235)[[symbols_strategies[0][0] + gf]] 
df_crossover2 = movingAverageCrossover(pd.DataFrame(dfPrices.loc[:, symbols_strategies[1][0]].copy()),
                                       22, 235)[[symbols_strategies[1][0] + gs]]
df_crossover3 = movingAverageCrossover(pd.DataFrame(dfPrices.loc[:, symbols_strategies[2][0]].copy()),
                                       22, 235)[[symbols_strategies[2][0] + gs]]
df_crossover4 = movingAverageCrossover(pd.DataFrame(dfPrices.loc[:, symbols_strategies[3][0]].copy()),
                                       87, 105)[[symbols_strategies[3][0] + gf]]
df_crossover5= movingAverageCrossover(pd.DataFrame(dfPrices.loc[:, symbols_strategies[4][0]].copy()),
                                      87, 105)[[symbols_strategies[4][0] + gs]] 
df_crossover6 = movingAverageCrossover(pd.DataFrame(dfPrices.loc[:, symbols_strategies[5][0]].copy()),
                                       87, 105)[[symbols_strategies[5][0] + gs]]
df_crossover7 = bollingerBands(dfPrices[['INTC']].copy(), 22, 2)[['INTC-BB']]
df_crossover8 = bollingerBands(dfPrices[['PFE']].copy(), 22, 2)[['PFE-BB']]

In [5]:
df_merged = pd.merge(df_crossover1, df_crossover2, left_index=True, right_index=True, how='inner')
df_merged = pd.merge(df_merged, df_crossover3, left_index=True, right_index=True, how='inner')
df_merged = pd.merge(df_merged, df_crossover4, left_index=True, right_index=True, how='inner')
df_merged = pd.merge(df_merged, df_crossover5, left_index=True, right_index=True, how='inner')
df_merged = pd.merge(df_merged, df_crossover6, left_index=True, right_index=True, how='inner')
df_merged = pd.merge(df_merged, df_crossover7, left_index=True, right_index=True, how='inner')
df_merged = pd.merge(df_merged, df_crossover8, left_index=True, right_index=True, how='inner')
df_merged = df_merged[1:]

In [6]:
print_pct_data(df_merged.head())

,AAPL-MAFlat,GE-MAShort,SPY-MAShort,FBNDX-MAFlat,XPT-MAShort,GBP-MAShort,INTC-BB,PFE-BB
Date,,,,,,,,
2000-12-05 00:00:00,0.0000%,-4.7290%,-3.2283%,0.4240%,-1.1263%,1.3524%,8.8907%,0.0000%
2000-12-06 00:00:00,-0.0000%,0.3470%,1.6127%,0.2817%,-2.1260%,-0.8206%,-12.5626%,-0.0000%
2000-12-07 00:00:00,0.0000%,0.8144%,1.3816%,0.1405%,1.4115%,-0.0623%,1.7561%,0.0000%
2000-12-08 00:00:00,0.0000%,-3.1055%,-0.2335%,-0.1405%,-0.7689%,-0.4833%,5.0906%,0.0000%
2000-12-11 00:00:00,0.0000%,-0.2262%,-3.4166%,0.0000%,1.0781%,-0.3438%,9.6312%,0.0000%


# 3. Making MVO weights portfolio

In [7]:
# Weights in our portfolio
weights = pd.Series({"AAPL-MAFlat": 0.146776,
                    "GE-MAShort": 0.071009,
                    "INTC-BB": 0.095478,
                    "PFE-BB": 0.141931,
                    "SPY-MAShort": 0.053683,
                    "FBNDX-MAFlat": 0.220983,
                    "XPT-MAShort": 0.143070,
                    "GBP-MAShort": 0.127069})

df_merged['Max Sharpe Portfolio'] = df_merged.mul(weights).sum(axis=1)
print_pct_data(df_merged.head())

,AAPL-MAFlat,GE-MAShort,SPY-MAShort,FBNDX-MAFlat,XPT-MAShort,GBP-MAShort,INTC-BB,PFE-BB,Max Sharpe Portfolio
Date,,,,,,,,,
2000-12-05 00:00:00,0.0000%,-4.7290%,-3.2283%,0.4240%,-1.1263%,1.3524%,8.8907%,0.0000%,0.4442%
2000-12-06 00:00:00,-0.0000%,0.3470%,1.6127%,0.2817%,-2.1260%,-0.8206%,-12.5626%,-0.0000%,-1.4344%
2000-12-07 00:00:00,0.0000%,0.8144%,1.3816%,0.1405%,1.4115%,-0.0623%,1.7561%,0.0000%,0.5248%
2000-12-08 00:00:00,0.0000%,-3.1055%,-0.2335%,-0.1405%,-0.7689%,-0.4833%,5.0906%,0.0000%,0.0505%
2000-12-11 00:00:00,0.0000%,-0.2262%,-3.4166%,0.0000%,1.0781%,-0.3438%,9.6312%,0.0000%,0.8306%


In [8]:
df_partA = df_merged.loc[df_merged.index <= '2018-12-31']
calcSummaryStatistics(df_partA, print_screen=True)
summarystats_partA = calcSummaryStatistics(df_partA, print_screen = False)

Summary Statistics for Returns:


,AAPL-MAFlat,GE-MAShort,SPY-MAShort,FBNDX-MAFlat,XPT-MAShort,GBP-MAShort,INTC-BB,PFE-BB,Max Sharpe Portfolio
Average Annual Return,25.48%,16.19%,10.16%,0.85%,13.00%,4.94%,11.06%,9.09%,10.45%
Annual Risk,28.00%,30.99%,19.33%,2.75%,22.18%,9.51%,25.98%,18.59%,7.31%
Sharpe Ratio,0.910103,0.522358,0.525463,0.308159,0.585966,0.518729,0.425637,0.488684,1.430187
maxDD,-1543.26%,-513.64%,-114.11%,-10.06%,-815.95%,-37.21%,-115.78%,-79.48%,-38.35%
Success Ratio,65.46%,52.10%,53.66%,80.59%,52.67%,51.40%,74.65%,74.10%,54.15%


In [9]:
df_partB = df_merged.loc[df_merged.index > '2018-12-31']
calcSummaryStatistics(df_partB, print_screen=True)
summarystats_partB = calcSummaryStatistics(df_partB, print_screen = False)

Summary Statistics for Returns:


,AAPL-MAFlat,GE-MAShort,SPY-MAShort,FBNDX-MAFlat,XPT-MAShort,GBP-MAShort,INTC-BB,PFE-BB,Max Sharpe Portfolio
Average Annual Return,21.10%,-24.02%,-4.94%,0.96%,-11.87%,-2.36%,14.11%,-13.84%,-1.28%
Annual Risk,29.59%,42.65%,21.87%,3.87%,29.39%,9.82%,31.94%,22.11%,10.61%
Sharpe Ratio,0.713252,-0.563180,-0.225847,0.247193,-0.403909,-0.240609,0.441817,-0.626079,-0.120391
maxDD,-87.77%,-87.94%,-51.20%,-9.48%,-59.82%,-18.74%,-49.26%,-64.92%,-23.77%
Success Ratio,63.18%,48.87%,53.39%,81.59%,49.37%,50.38%,71.21%,70.96%,51.55%


In [10]:
print(f'First sub-period Sharpe = {summarystats_partA.loc["Sharpe Ratio", "Max Sharpe Portfolio"]}')
print(f'Second sub-period Sharpe = {summarystats_partB.loc["Sharpe Ratio", "Max Sharpe Portfolio"]}')

First sub-period Sharpe = 1.4301873153464288
Second sub-period Sharpe = -0.12039141352543574


The Sharpe for the new out-of sample period (-0.1203914135254358) is less than the Part A first sub-period Sharpe (1.4301873153464288). 

# 4. SR of Equal Weighted Benchmark Strategy

In [11]:
df_crossover1 = movingAverageCrossover(dfPrices[['AAPL']].copy(), 22, 235)['AAPL-BMK']
df_crossover2 = movingAverageCrossover(dfPrices[['INTC']].copy(), 22, 235)['INTC-BMK']
df_crossover3 = movingAverageCrossover(dfPrices[['GE']].copy(), 22, 235)['GE-BMK']
df_crossover4 = movingAverageCrossover(dfPrices[['PFE']].copy(), 22, 235)['PFE-BMK']
df_crossover5 = movingAverageCrossover(dfPrices[['SPY']].copy(), 22, 235)['SPY-BMK']
df_crossover6 = movingAverageCrossover(dfPrices[['FBNDX']].copy(), 87, 105)['FBNDX-BMK']
df_crossover7 = movingAverageCrossover(dfPrices[['XPT']].copy(), 87, 105)['XPT-BMK']
df_crossover8 = movingAverageCrossover(dfPrices[['GBP']].copy(), 87, 105)['GBP-BMK']

df_bmk = pd.merge(df_crossover1, df_crossover2, left_index=True, right_index=True, how='inner')
df_bmk = pd.merge(df_bmk, df_crossover3, left_index=True, right_index=True, how='inner')
df_bmk = pd.merge(df_bmk, df_crossover4, left_index=True, right_index=True, how='inner')
df_bmk = pd.merge(df_bmk, df_crossover5, left_index=True, right_index=True, how='inner')
df_bmk = pd.merge(df_bmk, df_crossover6, left_index=True, right_index=True, how='inner')
df_bmk = pd.merge(df_bmk, df_crossover7, left_index=True, right_index=True, how='inner')
df_bmk = pd.merge(df_bmk, df_crossover8, left_index=True, right_index=True, how='inner')
df_bmk = df_bmk.loc[df_bmk.index > '2018-12-31']

# Calculate sharpe ratio, using matrix formulation of portfolio Return and Risk
equal_weight = np.array([1/df_bmk.shape[1]]*df_bmk.shape[1])
annualization_factor = 260

annual_sharpe = np.sqrt(annualization_factor) * np.dot(equal_weight, df_bmk.mean().T) / np.sqrt(np.dot(np.dot(equal_weight, df_bmk.cov()), equal_weight.T))
annual_sharpe

0.36319437828682327

In [12]:
df_bmk['Equal-weight Portfolio'] = df_bmk.mul(equal_weight).sum(axis=1)
calcSummaryStatistics(df_bmk, print_screen=True)
summarystats_BMK = calcSummaryStatistics(df_bmk, print_screen = False)

Summary Statistics for Returns:


,AAPL-BMK,INTC-BMK,GE-BMK,PFE-BMK,SPY-BMK,FBNDX-BMK,XPT-BMK,GBP-BMK,Equal-weight Portfolio
Average Annual Return,31.94%,-6.04%,19.34%,-4.83%,11.68%,-2.46%,2.87%,-0.97%,6.44%
Annual Risk,33.41%,39.72%,42.66%,27.11%,21.86%,5.73%,29.40%,9.82%,17.74%
Sharpe Ratio,0.956061,-0.152109,0.453289,-0.178135,0.534244,-0.429167,0.097659,-0.098605,0.363194
maxDD,-137.40%,-101.64%,-103.19%,-66.38%,-48.49%,-27.10%,-55.89%,-27.73%,-41.96%
Success Ratio,53.72%,50.46%,51.05%,48.70%,54.56%,59.16%,50.88%,49.62%,53.56%


The passive benchmark-only Sharpe ratio for the new out-of-sample period is 0.36319437828682327

# 5. Conclusion

The active portfolio initially demonstrated strong performance with the highest Sharpe ratio of 1.43 in the first sub-periods but faced a stark decline in the new out-of-sample period, resulting in a negative Sharpe ratio of -0.12. In contrast, the passive benchmark-only portfolio maintained a positive Sharpe ratio of 0.36 during the same out-of-sample period. This may challenge the robustness and adaptability of the active strategy to changing market conditions.

Comparing unfavorably to the passive benchmark during the out-of-sample period prompts a critical review of the active portfolio's risk management and model calibration. Assessing changes in risk exposure, volatility, and correlations is essential to understand and address the observed drop in performance. Recalibrating the model based on these insights becomes imperative for improving the strategy's resilience.

While the out-of-sample underperformance may be disappointing, it's not a futile exercise. Rather, it serves as a valuable learning opportunity to identify weaknesses and refine the active portfolio strategy. Lessons learned from this experience can guide adjustments, contributing to the continuous improvement of the strategy's effectiveness. There are other factors that contribute to this behavior as well. One of them being COVID, and the other factor is that the increase in the popularity of Energy companies (and hence its returns) in last 5 years. Hence a robust active strategy must account for changes in the market and also consider the change of basket of stocks in its implementation.

Additionally, reevaluation of the active portfolio's diversification and asset allocation is imperative. 

Concentration in certain assets or sectors may have played a role in the underperformance. Ensuring a well-diversified portfolio is crucial to mitigating risks and improving overall performance across various market conditions. 